In [3]:
import cv2
import mediapipe as mp
import tensorflow as tf

2024-07-08 15:22:41.418150: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 15:22:41.421248: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 15:22:41.431062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 15:22:41.451486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 15:22:41.451529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 15:22:41.464686: I tensorflow/core/platform/cpu_feature_guard.cc:

In [4]:
model = tf.keras.models.load_model('get_gesture_model/gesture_model.h5')

2024-07-08 15:22:42.960845: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-08 15:22:42.961375: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/benjamin/benjamin_external_ssd1/vg_motion_control/env_vg_motion_control/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

I0000 00:00:1720444963.060736   45396 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720444963.148757   45519 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 2060/PCIe/SSE2


In [6]:
running = True
cap = cv2.VideoCapture(0)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1720444963.162709   45509 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720444963.185039   45515 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
while running:
	success, frame = cap.read()
	if not success:
		break
	frame = cv2.flip(frame, 1)
	results = hands.process(frame)
	if results.multi_hand_landmarks:
		for hand, hand_landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
			print(hand.classification[0].label)
			index_finger_landmarks = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
			index_finger_x = int(index_finger_landmarks.x * frame.shape[1])
			index_finger_y = int(index_finger_landmarks.y * frame.shape[0])
			if index_finger_x < frame.shape[1] // 6:
				gesture_x = 'Left'
			elif index_finger_x < frame.shape[1] // 2 and index_finger_x > frame.shape[1] * 1 // 3:
				gesture_x = 'Right'
			else:
				gesture_x = 'Neutral'
			if index_finger_y < frame.shape[0] // 3:
				gesture_y = 'Up'
			elif index_finger_y > frame.shape[0] * 2 // 3:
				gesture_y = 'Down'
			else:
				gesture_y = 'Neutral'
			cv2.circle(frame, (index_finger_x, index_finger_y), 10, (0, 255, 0), -1)
			cv2.putText(frame, gesture_x + gesture_y, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
	cv2.imshow('Gesture Recognition', frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
	
cap.release()
cv2.destroyAllWindows()